In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score

In [2]:
# Importing the datasets
train = pd.read_csv('C:/Users/USER/train(excellent).csv')
test = pd.read_csv('C:/Users/USER/test(excellent).csv')
submission = pd.read_csv('C:/Users/USER/SampleSubmission(excellent).csv')

In [3]:
train["Store_Size"].replace(np.nan, "Medium", inplace=True)
test["Store_Size"].replace(np.nan, "Medium", inplace=True)
train["Item_Weight"].fillna(method='bfill', inplace=True)
test["Item_Weight"].fillna(method='bfill', inplace=True)

In [4]:
# Splitting into dependent and independent varaibles
X = train.drop(['Item_ID','Store_ID','Item_Store_ID','Item_Store_Returns','Item_Visibility','Item_Type','Store_Start_Year'], axis=1)
Y = train[['Item_Store_Returns']].values

In [5]:
test_given = test.drop(['Item_ID','Store_ID','Item_Store_ID','Item_Visibility','Item_Type','Store_Start_Year'], axis=1)

test_given = pd.get_dummies(test_given)
test_given = test_given.values

# Encoding categorical variables
X = pd.get_dummies(X)
X = X.values

In [6]:
X.reshape(-1,1)
Y.reshape(-1,1)

array([[  709.08],
       [ 6381.69],
       [ 6381.69],
       ...,
       [ 5685.93],
       [11778.  ],
       [17870.07]])

In [7]:
# splitting into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state = 0)

In [8]:
from sklearn.ensemble import GradientBoostingRegressor

In [9]:
# Predicting the given test set
reg = GradientBoostingRegressor()
reg.fit(X, Y.ravel())
Y_pred_test = reg.predict(test_given)

In [10]:
from sklearn.model_selection import cross_val_score
mse = cross_val_score(reg, X, Y.ravel(), scoring = 'neg_mean_squared_error', cv=20)
mse.mean()

-8699378.832496535

In [11]:
Y_pred_test = reg.predict(test_given)

Y_pred_test

array([1601.9512816 , 8118.71838415, 7735.49219933, ..., 9421.46357013,
       1386.68761995, 7766.58380949])

In [12]:
identifier = test['Item_Store_ID']
identifier

sub = pd.DataFrame(identifier, columns=['Item_Store_ID'])
sub['Item_Store_Returns'] = Y_pred_test
sub

,Item_Store_ID,Item_Store_Returns
0,DRA59_BABATUNJI010,1601.951282
1,DRA59_BABATUNJI013,8118.718384
2,DRB01_BABATUNJI013,7735.492199
3,DRB13_BABATUNJI010,1374.685841
4,DRB13_BABATUNJI013,7683.632659
...,...,...
3527,NCZ42_BABATUNJI010,1647.105783
3528,NCZ42_BABATUNJI013,9314.576813
3529,NCZ42_BABATUNJI049,9421.463570
3530,NCZ53_BABATUNJI010,1386.687620


In [13]:
# Predicting the given test set
params = {'n_estimators':3, 'max_depth':3, 'learning_rate':1, 'criterion': 'mse'}
reg = GradientBoostingRegressor(**params)
reg.fit(X_train, Y_train.ravel())

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [14]:
Y_pred = reg.predict(test_given)

Y_pred

array([ 857.37089969, 8035.26404054, 8035.26404054, ..., 8341.58734557,
       1150.40018103, 8035.26404054])

In [15]:
identifier1 = test['Item_Store_ID']
identifier1

sub1 = pd.DataFrame(identifier1, columns=['Item_Store_ID'])
sub1['Item_Store_Returns'] = Y_pred
sub1

,Item_Store_ID,Item_Store_Returns
0,DRA59_BABATUNJI010,857.370900
1,DRA59_BABATUNJI013,8035.264041
2,DRB01_BABATUNJI013,8035.264041
3,DRB13_BABATUNJI010,1150.400181
4,DRB13_BABATUNJI013,8035.264041
...,...,...
3527,NCZ42_BABATUNJI010,3488.898074
3528,NCZ42_BABATUNJI013,10373.761934
3529,NCZ42_BABATUNJI049,8341.587346
3530,NCZ53_BABATUNJI010,1150.400181


In [16]:
sub.to_csv('submission1.csv', index=False)

In [17]:
import xgboost as xgb

In [18]:
xgb_params = {'eta':0.04, 'max_depth':6, 'subsample':0.80, 'objective': 'reg:linear', 'scoring': 'neg_mean_absolute_error','lambda':0.8, 'alpha': 0.4,'silent':1, 'min_child_weight':5}

In [19]:
data_dmatrix = xgb.DMatrix()


dtrain = xgb.DMatrix(X_train,Y_train)
dtest = xgb.DMatrix(X_test)
dtest_given = xgb.DMatrix(test_given)
num_boost_rounds = 250

TypeError: __init__() missing 1 required positional argument: 'data'

In [ ]:
model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round = num_boost_rounds)

In [ ]:
xgb_pred1 = model.predict(dtest_given)
xgb_pred = model.predict(test)

In [ ]:
xgb_pred1

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
rmse = mean_squared_error(Y_test,xgb_pred1)
rmse